# 준비

## 라이브러리

In [20]:
import pandas as pd
import numpy as np
import scipy as sp

import missingno as msno

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

pd.options.display.max_columns = 50 # 칼럼수 50개까지

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder, StandardScaler, RobustScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.model_selection import cross_val_score
import xgboost as xgb
import lightgbm as lgb
from bayes_opt import BayesianOptimization

from tqdm import tqdm

RANDOM_SEED = 42

## 데이터

In [2]:
train_df = pd.read_csv("kaggle-kakr-housing-data/train.csv")
test_df = pd.read_csv("kaggle-kakr-housing-data/test.csv")
sub = pd.read_csv("kaggle-kakr-housing-data/sample_submission.csv")

1. ID : 집을 구분하는 번호
2. date : 집을 구매한 날짜
3. price : 타겟 변수인 집의 가격
4. bedrooms : 침실의 수
5. bathrooms : 침실당 화장실 개수
6. sqft_living : 주거 공간의 평방 피트
7. sqft_lot : 부지의 평방 피트
8. floors : 집의 층수
9. waterfront : 집의 전방에 강이 흐르는지 유무 (a.k.a. 리버뷰)
10. view : 집이 얼마나 좋아 보이는지의 정도
11. condition : 집의 전반적인 상태
12. grade : King County grading 시스템 기준으로 매긴 집의 등급
13. sqft_above : 지하실을 제외한 평방 피트
14. sqft_basement : 지하실의 평방 피트
15. yr_built : 집을 지은 년도
16. yr_renovated : 집을 재건축한 년도
17. zipcode : 우편번호
18. lat : 위도
19. long : 경도
20. sqft_living15 : 근처 15 가구의 주거 공간, 평방 피트
21. sqft_lot15 : 근처 15가구의 부지, 평방 피트

In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15035 entries, 0 to 15034
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             15035 non-null  int64  
 1   date           15035 non-null  object 
 2   price          15035 non-null  float64
 3   bedrooms       15035 non-null  int64  
 4   bathrooms      15035 non-null  float64
 5   sqft_living    15035 non-null  int64  
 6   sqft_lot       15035 non-null  int64  
 7   floors         15035 non-null  float64
 8   waterfront     15035 non-null  int64  
 9   view           15035 non-null  int64  
 10  condition      15035 non-null  int64  
 11  grade          15035 non-null  int64  
 12  sqft_above     15035 non-null  int64  
 13  sqft_basement  15035 non-null  int64  
 14  yr_built       15035 non-null  int64  
 15  yr_renovated   15035 non-null  int64  
 16  zipcode        15035 non-null  int64  
 17  lat            15035 non-null  float64
 18  long  

## helper

In [2]:
def show_price_corr(data):
    df = data
    df = df[~df["price"].isna()]
    cor_abs = abs(df.corr(method='spearman')) 
    cor_cols = cor_abs.nlargest(n=10, columns='price').index
    cor = np.array(sp.stats.spearmanr(df[cor_cols].values))[0] # 10 x 10
    print(cor_cols.values)
    plt.figure(figsize=(8,8))
    sns.set(font_scale=1)
    sns.heatmap(cor, fmt='.2f', annot=True, square=True , annot_kws={'size' : 6} ,xticklabels=cor_cols.values, yticklabels=cor_cols.values)
    plt.show()

In [3]:
def rmse_exp(y_true, y_pred):
    return np.sqrt(mean_squared_error(np.expm1(y_true), np.expm1(y_pred)))

In [4]:
def TTS(data, do_ohe=True):
    df = data.drop(['id','price','data'], axis=1).copy()
    cat_cols = df.select_dtypes('object').columns
    for col in cat_cols:
        if do_ohe:
            ohe_df = pd.get_dummies(df[[col]], prefix='ohe_'+col)
            df.drop(col, axis=1, inplace=True)
            df = pd.concat([df, ohe_df], axis=1)
        else:
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col])

    train_len = data[data['data'] == 'train'].shape[0]
    X_train = df.iloc[:train_len]
    X_test = df.iloc[train_len:]
    y_train = data[data['data'] == 'train']['price']
    
    return X_train, X_test, y_train

def get_oof_lgb(X_train, y_train, X_test, lgb_param, verbose_eval=False, return_cv_score_only=False):

    folds = KFold(n_splits=5, shuffle=True, random_state=RANDOM_SEED)
    oof = np.zeros(len(X_train))
    predictions = np.zeros(len(X_test))
    feature_importance_df = pd.DataFrame()
    params = {'verbose': -1}
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train.values, y_train.values)):
        if verbose_eval > 0: print(f'Fold : {fold_ + 1}')
        trn_data = lgb.Dataset(X_train.iloc[trn_idx], label=y_train.iloc[trn_idx], params=params)
        val_data = lgb.Dataset(X_train.iloc[val_idx], label=y_train.iloc[val_idx])

        num_round = 100000
        clf = lgb.train(lgb_param, trn_data, num_round, valid_sets=[trn_data, val_data],
                        verbose_eval=verbose_eval, early_stopping_rounds=200)
        oof[val_idx] = clf.predict(X_train.iloc[val_idx], num_iteration=clf.best_iteration)
        predictions += clf.predict(X_test, num_iteration=clf.best_iteration) / folds.n_splits
        
        cv_fold_score = rmse_exp(y_train.iloc[val_idx], oof[val_idx])
        
        if verbose_eval > 0: print(f'Fold {fold_ + 1} / CV-Score: {cv_fold_score:.6f}')
        
        fold_importance_df = pd.DataFrame()
        fold_importance_df['feature'] = X_train.columns.tolist()
        fold_importance_df['importance'] = clf.feature_importance('gain')
        fold_importance_df['fold'] = fold_ + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    cv_score = rmse_exp(y_train, oof)
    print(f'CV-Score: {cv_score:.6f}')
    if return_cv_score_only: return cv_score
    else: return oof, predictions, cv_score, feature_importance_df
    
def plot_feature_importance(fi_df, num_feature=20):
    cols = (fi_df[['feature', 'importance']]
            .groupby('feature')
            .mean()
            .sort_values(by='importance', ascending=False)[:num_feature].index)
    best_features = fi_df.loc[fi_df.feature.isin(cols)]

    sns.barplot(x='importance', y='feature', data=best_features.sort_values(by='importance', ascending=False))
    plt.title('Feature Importances (averaged over folds)')
    plt.tight_layout()
    plt.show()
    
def plot_numeric_for_regression(df, field, target_field='price'):
    df = df[df[field].notnull()]

    fig = plt.figure(figsize = (16, 7))
    ax1 = plt.subplot(121)
    
    sns.distplot(df[df['data'] == 'train'][field], label='Train', hist_kws={'alpha': 0.5}, ax=ax1)
    sns.distplot(df[df['data'] == 'test'][field], label='Test', hist_kws={'alpha': 0.5}, ax=ax1)

    plt.xlabel(field)
    plt.ylabel('Density')
    plt.legend()
    
    ax2 = plt.subplot(122)
    
    df_copy = df[df['data'] == 'train'].copy()

    sns.scatterplot(x=field, y=target_field, data=df_copy, ax=ax2)
    
    plt.show()
    
def plot_categorical_for_regression(df, field, target_field='price', show_missing=True, missing_value='NA'):
    df_copy = df.copy()
    if show_missing: df_copy[field] = df_copy[field].fillna(missing_value)
    df_copy = df_copy[df_copy[field].notnull()]

    ax1_param = 121
    ax2_param = 122
    fig_size = (16, 7)
    if df_copy[field].nunique() > 30:
        ax1_param = 211
        ax2_param = 212
        fig_size = (16, 10)
    
    fig = plt.figure(figsize = fig_size)
    ax1 = plt.subplot(ax1_param)
    
    sns.countplot(x=field, hue='data', order=np.sort(df_copy[field].unique()), data=df_copy)
    plt.xticks(rotation=90, fontsize=11)
    
    ax2 = plt.subplot(ax2_param)
    
    df_copy = df_copy[df_copy['data'] == 'train']

    sns.boxplot(x=field, y=target_field, data=df_copy, order=np.sort(df_copy[field].unique()), ax=ax2)
    plt.xticks(rotation=90, fontsize=11)
    
    plt.show()
    
def load_original_data():
    train = pd.read_csv('../input/train.csv')
    test = pd.read_csv('../input/test.csv')

    train_copy = train.copy()
    train_copy['data'] = 'train'
    test_copy = test.copy()
    test_copy['data'] = 'test'
    test_copy['price'] = np.nan

    # remove outlier
    train_copy = train_copy[~((train_copy['sqft_living'] > 12000) & (train_copy['price'] < 3000000))].reset_index(drop=True)

    # concat train, test data to preprocess
    data = pd.concat([train_copy, test_copy], sort=False).reset_index(drop=True)
    data = data[train_copy.columns]

    data.drop('date', axis=1, inplace=True)
    data['zipcode'] = data['zipcode'].astype(str)

    # fix skew feature
    skew_columns = ['price']

    for c in skew_columns:
        data[c] = np.log1p(data[c])
        
    return data

# 전처리

In [164]:
# 데이터 합치기
train_df["data"]="train"
test_df["data"]="test"
df = pd.concat([train_df, test_df], axis=0)
df.head(3)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,data
0,0,20141013T000000,221900.0,3,1.0,1180,5650,1.0,0,0,3,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650,train
1,1,20150225T000000,180000.0,2,1.0,770,10000,1.0,0,0,3,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062,train
2,2,20150218T000000,510000.0,3,2.0,1680,8080,1.0,0,0,3,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503,train


In [165]:
# price
df["price"] = df["price"].apply(lambda x: np.log1p(x))

In [166]:
# date
df["date"] = df["date"].apply(lambda x: str(x)[:8])
df["year"] = df["date"].apply(lambda x: x[:4])
df["month"] = df["date"].apply(lambda x: int(x[4:6]))

In [167]:
# backup
df = df.reset_index(drop=True)
original_df = df.copy()

In [175]:
# basemodel
X_train, X_test, y_train = TTS(df)
print(X_train.shape, X_test.shape)

lgb_param = {
    'objective': 'regression',
    'learning_rate': 0.05,
    'num_leaves': 15,
    'bagging_fraction': 0.7,
    'bagging_freq': 1,
    'feature_fraction': 0.7,
    'seed': RANDOM_SEED,
    'metric': ['rmse'],
    'verbose': -1,
}

oof, pred, cv_score, fi_df = get_oof_lgb(X_train, y_train, X_test, lgb_param)

(15035, 394) (6468, 394)
CV-Score: 119571.897666


In [178]:
# live / lot False
df = original_df.copy()
df["live_per_lot"] = df["sqft_living"] / df["sqft_lot"]
X_train, X_test, y_train = TTS(df)
oof, pred, cv_score, fi_df = get_oof_lgb(X_train, y_train, X_test, lgb_param)

CV-Score: 119571.897666


In [185]:
# renovated
df = original_df.copy()
last_built = []
for idx, row in df.iterrows():
    if row["yr_built"] < row["yr_renovated"]:
        last_built.append(row["yr_renovated"])
    else:
        last_built.append(row["yr_built"])
       
df["last_built"] = last_built
df = df.drop(["yr_built","yr_renovated"], axis=1)
X_train, X_test, y_train = TTS(df)
oof, pred, cv_score, fi_df = get_oof_lgb(X_train, y_train, X_test, lgb_param)

CV-Score: 118434.275475


In [177]:
# yr_built
df = original_df.copy()
df["yr_built"] = 2015-df["yr_built"]
X_train, X_test, y_train = TTS(df)
oof, pred, cv_score, fi_df = get_oof_lgb(X_train, y_train, X_test, lgb_param)

CV-Score: 117817.094319


In [54]:
# zipcode seperate
df = original_df.copy()

df["zipcode"] = df["zipcode"].apply(lambda x: str(x)[2:])
df["zipcode-1"] = df["zipcode"].apply(lambda x: "zip_"+str(x)[:1])
df["zipcode-2"] = df["zipcode"].apply(lambda x: "zip_"+str(x)[1:2])
df["zipcode-3"] = df["zipcode"].apply(lambda x: "zip_"+str(x)[2:])

X_train, X_test, y_train = TTS(df)
oof, pred, cv_score, fi_df = get_oof_lgb(X_train, y_train, X_test, lgb_param)

CV-Score: 116842.144618


In [55]:
# zipcode combination
df["zipcode-12"] = df["zipcode"].apply(lambda x: "zip_"+str(x)[:2])
df["zipcode-13"] = df["zipcode"].apply(lambda x: "zip_"+str(x)[:1]+str(x)[2:])
df["zipcode-23"] = df["zipcode"].apply(lambda x: "zip_"+str(x)[1:])

X_train, X_test, y_train = TTS(df)
oof, pred, cv_score, fi_df = get_oof_lgb(X_train, y_train, X_test, lgb_param)

CV-Score: 116415.573613


In [53]:
# lat long - PCA component:1 <=
df = original_df.copy()

coord = df[['lat','long']]
pca = PCA(n_components=1)
pca.fit(coord)
coord_pca = pca.transform(coord)
df['coord_pca1'] = coord_pca[:, 0]

X_train, X_test, y_train = TTS(df)
oof, pred, cv_score, fi_df = get_oof_lgb(X_train, y_train, X_test, lgb_param)

CV-Score: 116249.246033


In [52]:
# lat long - PCA component: 2 => False
df = original_df.copy()

coord = df[['lat','long']]
pca = PCA(n_components=2)
pca.fit(coord)
coord_pca = pca.transform(coord)
df['coord_pca1'] = coord_pca[:, 0]
df['coord_pca2'] = coord_pca[:, 1]

X_train, X_test, y_train = TTS(df)
oof, pred, cv_score, fi_df = get_oof_lgb(X_train, y_train, X_test, lgb_param)

CV-Score: 117021.270829


In [65]:
# lat long - Kmeans K:39 <=
df = original_df.copy()

k_range = range(3, 50)

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=RANDOM_SEED).fit(coord)
    coord_cluster = kmeans.predict(coord)
    df['coord_cluster'] = coord_cluster
    df['coord_cluster'] = df['coord_cluster'].map(lambda x: str(x).rjust(2, '0'))
    
    X_train, X_test, y_train = TTS(df)

    print('K :', k)
    get_oof_lgb(X_train, y_train, X_test, lgb_param)
    print()

K : 3
CV-Score: 115380.173709

K : 4
CV-Score: 116272.473179

K : 5
CV-Score: 116677.051501

K : 6
CV-Score: 116172.262792

K : 7
CV-Score: 114936.908439

K : 8
CV-Score: 115856.724413

K : 9
CV-Score: 116398.236962

K : 10
CV-Score: 115772.735695

K : 11
CV-Score: 118542.069559

K : 12
CV-Score: 116696.261773

K : 13
CV-Score: 117202.111614

K : 14
CV-Score: 116946.552685

K : 15
CV-Score: 117844.281331

K : 16
CV-Score: 118026.728299

K : 17
CV-Score: 117036.451056

K : 18
CV-Score: 116229.235762

K : 19
CV-Score: 115908.161469

K : 20
CV-Score: 116367.811538

K : 21
CV-Score: 116443.820134

K : 22
CV-Score: 116884.102800

K : 23
CV-Score: 116997.806480

K : 24
CV-Score: 116546.456531

K : 25
CV-Score: 117445.533713

K : 26
CV-Score: 116033.046960

K : 27
CV-Score: 115849.854419

K : 28
CV-Score: 117088.229659

K : 29
CV-Score: 116761.047245

K : 30
CV-Score: 117584.581981

K : 31
CV-Score: 116544.435343

K : 32
CV-Score: 116197.244265

K : 33
CV-Score: 115619.382725

K : 34
CV-Score

In [138]:
# haversine distance 
def haversine_array(lat1, lng1, lat2, lng2): 
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2)) 
    AVG_EARTH_RADIUS = 6371 # in km 
    lat = lat2 - lat1 
    lng = lng2 - lng1 
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2 
    h = 2 * AVG_EARTH_RADIUS * np.arcsin(np.sqrt(d)) 
    return h

In [198]:
# distence 0.5
df = original_df.copy()
last_built = []
for idx, row in df.iterrows():
    if row["yr_built"] < row["yr_renovated"]:
        last_built.append(row["yr_renovated"])
    else:
        last_built.append(row["yr_built"])

df["last_built"] = last_built
df = df.drop(["yr_built","yr_renovated"], axis=1)

count10_list=[]
condi10_list=[]
grad10_list=[]
yr_built10_list = []

print("distance : ",dist*2)
for i in tqdm(range(len(df))):
    lat2 = df['lat'].values
    long2 = df['long'].values

    lat1 = df.loc[i, 'lat'] # id = 0 house lat
    long1 = df.loc[i, 'long'] # id = 0 house long
    dist_arr = haversine_array(lat1, long1, lat2, long2)
    neighbor_df = pd.DataFrame({
        'id': np.tile(np.array([df.loc[i, 'id']]), df.shape[0]),
        'neighbor_id': df['id'],
        'neighbor_lat': lat2,
        'neighbor_long': long2,
        'distance': dist_arr,
    })

    condi10_list.append(df["condition"][neighbor_df["distance"]<0.5].mean())
    grad10_list.append(df["grade"][neighbor_df["distance"]<0.5].mean())
    yr_built10_list.append(df["condition"][neighbor_df["distance"]<0.5].mean())
    count10_list.append(sum(neighbor_df["distance"]<=0.5))
df[f"count_1km"]=count10_list
df[f"conditon_1km"]=condi10_list
df[f"grade_1km"]=grad10_list
df[f"yr_built_1km"]=yr_built10_list

distance :  10.0


100%|████████████████████████████████████████████████████████████████████████████| 21503/21503 [00:58<00:00, 366.60it/s]


In [206]:
# 1km house count
temp = df.drop(["yr_built_1km","conditon_1km","grade_1km"],axis=1)
X_train, X_test, y_train = TTS(temp)

oof, pred, cv_score, fi_df = get_oof_lgb(X_train, y_train, X_test, lgb_param)

CV-Score: 117639.268041


In [203]:
# 1km yr_lastbuilt mean => False
temp = df.drop(["count_1km","conditon_1km","grade_1km"],axis=1)
X_train, X_test, y_train = TTS(temp)

oof, pred, cv_score, fi_df = get_oof_lgb(X_train, y_train, X_test, lgb_param)

CV-Score: 120004.360741


In [204]:
# 1km grade mean
temp = df.drop(["count_1km","conditon_1km","yr_built_1km"],axis=1)
X_train, X_test, y_train = TTS(temp)

oof, pred, cv_score, fi_df = get_oof_lgb(X_train, y_train, X_test, lgb_param)

CV-Score: 116922.818722


In [205]:
# 1km condition mean => False
temp = df.drop(["count_1km","yr_built_1km","grade_1km"],axis=1)
X_train, X_test, y_train = TTS(temp)

oof, pred, cv_score, fi_df = get_oof_lgb(X_train, y_train, X_test, lgb_param)

CV-Score: 120004.360741


In [207]:
distance_1km = df[["count_1km","grade_1km"]].copy()

In [212]:
# all

# last_built
df = original_df.copy()
last_built = []
for idx, row in df.iterrows():
    if row["yr_built"] < row["yr_renovated"]:
        last_built.append(row["yr_renovated"])
    else:
        last_built.append(row["yr_built"])
       
df["last_built"] = last_built
df = df.drop(["yr_built","yr_renovated"], axis=1)

# 2015 - built 
df["last_built"] = 2015-df["last_built"]

# zipcode
df["zipcode"] = df["zipcode"].apply(lambda x: str(x)[2:])
df["zipcode-1"] = df["zipcode"].apply(lambda x: "zip_"+str(x)[:1])
df["zipcode-2"] = df["zipcode"].apply(lambda x: "zip_"+str(x)[1:2])
df["zipcode-3"] = df["zipcode"].apply(lambda x: "zip_"+str(x)[2:])
df["zipcode-12"] = df["zipcode"].apply(lambda x: "zip_"+str(x)[:2])
df["zipcode-13"] = df["zipcode"].apply(lambda x: "zip_"+str(x)[:1]+str(x)[2:])
df["zipcode-23"] = df["zipcode"].apply(lambda x: "zip_"+str(x)[1:])

# lat long : PCA
coord = df[['lat','long']]
pca = PCA(n_components=1)
pca.fit(coord)
coord_pca = pca.transform(coord)
df['coord_pca1'] = coord_pca[:, 0]

# lat long : K means
kmeans = KMeans(n_clusters=39, random_state=RANDOM_SEED).fit(coord)
coord_cluster = kmeans.predict(coord)
df['coord_cluster'] = coord_cluster
df['coord_cluster'] = df['coord_cluster'].map(lambda x: str(x).rjust(2, '0'))

# distance 1km : haversine distance 
df["count_1km"]=distance_1km["count_1km"]
df["grade_1km"]=distance_1km["grade_1km"]

In [215]:
df.to_csv("kaggle-kakr-housing-data/prep_df.csv",index=False)

In [213]:
# all
X_train, X_test, y_train = TTS(df)
oof, pred, cv_score, fi_df = get_oof_lgb(X_train, y_train, X_test, lgb_param)

CV-Score: 115337.970625


# 모델링

In [6]:
df = pd.read_csv("kaggle-kakr-housing-data/prep_df.csv")
X_train, X_test, y_train = TTS(df)

In [23]:
pbounds = { 'learning_rate': (0.0005, 0.2),
            'n_estimators': (16, 1024),
            'max_depth': (3,10),   
            'subsample': (0.5,1), 
            'colsample_bytree': (0.5,1),   
            'num_leaves': (2,16),
            'min_child_weight': (1, 10)}


def lgbm_opt(learning_rate, n_estimators, max_depth, subsample, colsample_bytree, num_leaves, min_child_weight):

    params = {
        'learning_rate': learning_rate,
        'n_estimators' : int(round(n_estimators)),
        'max_depth' : int(round(max_depth)),
        'subsample': subsample,
        'colsample_bytree' : colsample_bytree,
        'num_leaves' : int(round(num_leaves)),
        'min_child_weight' : min_child_weight,
        'n_jobs' : -1
    }
    
    lgbm = lgb.LGBMRegressor(**params)
    
    kf = KFold(n_splits=5 , shuffle=True, random_state=50)

    score = cross_val_score(lgbm, X_train, y_train, scoring='neg_mean_squared_error', cv=kf, n_jobs=-1)
    
    return np.mean(score)
                   
BO_lgbm = BayesianOptimization(f = lgbm_opt, pbounds = pbounds, random_state=RANDOM_SEED, verbose=2) 

BO_lgbm.maximize(init_points=10, n_iter = 200)

BO_lgbm.max

params = BO_lgbm.max['params']
params['max_depth'] = int(round(params['max_depth']))
params['n_estimators'] = int(round(params['n_estimators']))
params['num_leaves'] = int(round(params['num_leaves']))

lgbm = lgb.LGBMRegressor(seed = RANDOM_SEED, **params)
lgbm.fit(X_train, y_train)


pred_lgbm = np.expm1(lgbm.predict(X_test))

|   iter    |  target   | colsam... | learni... | max_depth | min_ch... | n_esti... | num_le... | subsample |
-------------------------------------------------------------------------------------------------------------
| 1         | -0.03241  | 0.6873    | 0.1902    | 8.124     | 6.388     | 173.3     | 4.184     | 0.529     |
| 2         | -0.02667  | 0.9331    | 0.1204    | 7.957     | 1.185     | 993.7     | 13.65     | 0.6062    |
| 3         | -0.03252  | 0.5909    | 0.03709   | 5.13      | 5.723     | 451.4     | 6.077     | 0.8059    |
| 4         | -0.02819  | 0.5697    | 0.05878   | 5.565     | 5.105     | 807.5     | 4.795     | 0.7571    |
| 5         | -0.1208   | 0.7962    | 0.009767  | 7.253     | 2.535     | 81.57     | 15.28     | 0.9828    |
| 6         | -0.03317  | 0.9042    | 0.06127   | 3.684     | 7.158     | 459.7     | 3.709     | 0.7476    |
| 7         | -0.02705  | 0.5172    | 0.1819    | 4.811     | 6.963     | 330.2     | 9.281     | 0.7734    |
| 8       

| 45        | -0.02688  | 0.8431    | 0.2       | 10.0      | 10.0      | 720.9     | 7.895     | 0.8434    |
| 46        | -0.02661  | 0.5       | 0.1249    | 10.0      | 10.0      | 348.3     | 16.0      | 0.8676    |
| 47        | -0.03395  | 0.8211    | 0.2       | 3.0       | 10.0      | 629.4     | 2.0       | 1.0       |
| 48        | -0.02764  | 0.7243    | 0.1906    | 9.963     | 1.038     | 621.1     | 15.57     | 0.6246    |
| 49        | -0.02985  | 0.7157    | 0.0251    | 9.899     | 1.08      | 390.5     | 14.7      | 0.6175    |
| 50        | -0.03771  | 1.0       | 0.1584    | 10.0      | 10.0      | 428.5     | 2.0       | 0.5       |
| 51        | -0.2544   | 0.5       | 0.0005    | 3.0       | 10.0      | 311.3     | 2.0       | 1.0       |
| 52        | -0.02628  | 0.9678    | 0.07114   | 9.193     | 1.854     | 979.0     | 15.47     | 0.8345    |
| 53        | -0.2648   | 1.0       | 0.0005    | 3.0       | 10.0      | 157.9     | 2.0       | 0.8381    |
| 54      

| 90        | -0.02662  | 0.9692    | 0.09848   | 4.928     | 6.57      | 851.2     | 5.668     | 0.9325    |
| 91        | -0.02638  | 0.9235    | 0.08543   | 7.914     | 6.077     | 838.8     | 8.278     | 0.9118    |
| 92        | -0.02698  | 0.6903    | 0.1562    | 5.514     | 8.97      | 817.0     | 13.14     | 0.7186    |
| 93        | -0.02772  | 1.0       | 0.1679    | 10.0      | 3.258     | 805.6     | 16.0      | 0.9561    |
| 94        | -0.02621  | 0.6106    | 0.09897   | 3.342     | 8.65      | 829.5     | 15.59     | 0.884     |
| 95        | -0.02684  | 0.894     | 0.08103   | 9.885     | 2.986     | 417.0     | 15.84     | 0.5397    |
| 96        | -0.02623  | 0.7354    | 0.1119    | 3.555     | 1.322     | 843.6     | 15.93     | 0.7576    |
| 97        | -0.02749  | 0.9135    | 0.1785    | 7.366     | 8.616     | 651.1     | 15.16     | 0.915     |
| 98        | -0.02683  | 0.9645    | 0.1611    | 8.485     | 1.313     | 650.5     | 4.633     | 0.6012    |
| 99      

| 135       | -0.03377  | 0.5       | 0.2       | 3.0       | 1.0       | 658.1     | 2.0       | 0.5       |
| 136       | -0.2363   | 1.0       | 0.0005    | 10.0      | 1.0       | 571.9     | 2.0       | 1.0       |
| 137       | -0.02663  | 1.0       | 0.2       | 3.0       | 9.22      | 741.8     | 16.0      | 0.5       |
| 138       | -0.1015   | 0.8237    | 0.002931  | 9.689     | 5.662     | 423.0     | 8.217     | 0.5088    |
| 139       | -0.02669  | 0.7459    | 0.1912    | 3.366     | 1.816     | 723.8     | 6.795     | 0.7454    |
| 140       | -0.0267   | 0.6304    | 0.06885   | 3.205     | 9.204     | 965.9     | 6.351     | 0.9599    |
| 141       | -0.02766  | 1.0       | 0.2       | 10.0      | 10.0      | 964.8     | 11.81     | 1.0       |
| 142       | -0.02652  | 0.5       | 0.1935    | 10.0      | 7.282     | 870.9     | 6.492     | 1.0       |
| 143       | -0.02771  | 1.0       | 0.2       | 10.0      | 1.0       | 407.3     | 16.0      | 0.5       |
| 144     

| 180       | -0.02623  | 0.5       | 0.2       | 3.0       | 10.0      | 733.8     | 9.232     | 0.5       |
| 181       | -0.02622  | 0.5       | 0.2       | 3.0       | 10.0      | 823.7     | 9.17      | 0.5       |
| 182       | -0.0351   | 0.9677    | 0.1038    | 10.0      | 1.661     | 949.6     | 2.0       | 0.9112    |
| 183       | -0.02665  | 1.0       | 0.2       | 3.0       | 2.081     | 630.1     | 16.0      | 1.0       |
| 184       | -0.02666  | 1.0       | 0.2       | 3.0       | 10.0      | 624.9     | 16.0      | 1.0       |
| 185       | -0.0265   | 0.9458    | 0.07578   | 9.013     | 9.957     | 834.1     | 10.95     | 0.7129    |
| 186       | -0.0261   | 0.5534    | 0.07009   | 9.938     | 1.618     | 878.0     | 15.09     | 0.5714    |
| 187       | -0.02874  | 0.6909    | 0.04219   | 3.293     | 1.287     | 773.5     | 7.679     | 0.9264    |
| 188       | -0.02696  | 0.8082    | 0.1266    | 9.069     | 9.659     | 327.2     | 15.82     | 0.8198    |
| 189     

In [24]:
pred_lgbm

array([ 516412.82681629,  484104.37714833, 1313749.20186284, ...,
        464876.10463881,  322488.57858796,  435345.49672596])

In [26]:
sub = pd.read_csv("kaggle-kakr-housing-data/sample_submission.csv")
sub["price"] = pred_lgbm
sub.to_csv("submission_last2.csv",index=False)